In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
sns.set_style()

In [2]:
data = pd.read_csv('../input/crop-recommendation-dataset/Crop_recommendation.csv')
data.drop(['N', 'P', 'K', 'ph'], axis=1, inplace=True)
data.head()

,temperature,humidity,rainfall,label
0,20.879744,82.002744,202.935536,rice
1,21.770462,80.319644,226.655537,rice
2,23.004459,82.320763,263.964248,rice
3,26.491096,80.158363,242.864034,rice
4,20.130175,81.604873,262.717340,rice


In [3]:
data['label'].unique()

array(['rice', 'maize', 'chickpea', 'kidneybeans', 'pigeonpeas',
       'mothbeans', 'mungbean', 'blackgram', 'lentil', 'pomegranate',
       'banana', 'mango', 'grapes', 'watermelon', 'muskmelon', 'apple',
       'orange', 'papaya', 'coconut', 'cotton', 'jute', 'coffee'],
      dtype=object)

## Data Preprocessing

In [4]:
def convert_label(row, label):
    if row['label'] == label:
        row['label'] = 1
    else:
        row['label'] = 0
    return row

def take_200(data, label):
    
    label = label
    # Data for particular label
    specific_data = data[data['label'] == label]
    
    n = len(specific_data)
    
    # Data for beside particular label
    beside_data = data[data['label'] != label].sample(n)
    
    # Combine specific_data and beside_data
    combined_data = pd.concat([specific_data, beside_data])
    
    # Apply combined_data to convert_label function
    new_data = combined_data.apply(convert_label, label=label, axis=1)
    
    # Reset index and shuffle
    return new_data.sample(frac=1).reset_index(drop=True)

In [5]:
take_200(data, 'rice')

,temperature,humidity,rainfall,label
0,26.185524,96.966379,135.418622,0
1,23.641248,80.285979,263.110330,1
2,25.157455,83.117135,231.384316,1
3,24.488084,90.836872,103.196934,0
4,26.088679,80.379799,182.904350,1
...,...,...,...,...
195,20.879744,82.002744,202.935536,1
196,25.850370,81.955805,119.085617,0
197,26.730724,81.785968,280.404439,1
198,27.698193,51.415932,100.772070,0
